In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 15161972
paper_name = 'askree_mceachern_2004' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [5]:
original_data = pd.read_excel('raw_data/01263Table3.xlsx', sheet_name='Sheet1', skiprows=1)

In [6]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 183 x 5


In [7]:
original_data.head()

,ORF name,Common name,Measurement 1,Measurement 2,Comments
0,YLR233C,EST1,-293.3,NaN,NaN
1,YBL088C,TEL1,-240,NaN,NaN
2,YNL250W,RAD50,-233.2,NaN,NaN
3,YMR106C,YKU80,-227,NaN,NaN
4,YDR369C,XRS2,-225,NaN,NaN


In [8]:
original_data['orf'] = original_data['ORF name'].astype(str)

In [9]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [10]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [11]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

                                                      ORF name Common name  \
index_input                                                                  
181                                                        NaN         NaN   
182          Measured length differences of the Y' containi...         NaN   

            Measurement 1  Measurement 2 Comments  \
index_input                                         
181                   NaN            NaN      NaN   
182                   NaN            NaN      NaN   

                                                           orf  
index_input                                                     
181                                                        NAN  
182          MEASUREDLENGTHDIFFERENCESOFTHEYCONTAININGTELOM...  


In [12]:
original_data = original_data.loc[t,:]

In [13]:
original_data.set_index('orf', inplace=True)

In [14]:
original_data = original_data[['Measurement 1','Measurement 2']].apply(pd.to_numeric, axis=1, errors='coerce').mean(axis=1).to_frame()

In [15]:
original_data = original_data.groupby(original_data.index).mean()

In [16]:
original_data.shape

(181, 1)

# Load & process tested strains

In [17]:
# tested = pd.read_excel('raw_data/S.cerftpmata.xlsx', sheet_name='Sheet1', skiprows=2)

In [18]:
# tested.head()

In [19]:
# tested['orf'] = tested['ORF name'].astype(str)

In [20]:
# tested['orf'] = clean_orf(tested['orf'])

In [21]:
# typo_fixes = {'YOLO57W':'YOL057W','YOLO62C':'YOL062C','YKLO72W':'YKL072W','YJL206-A':'YJL206C-A','YLR287-A':'YLR287C-A'}
# tested['orf'] = tested['orf'].apply(lambda x: typo_fixes[x] if x in typo_fixes.keys() else x)

In [22]:
# tested['orf'] = translate_sc(tested['orf'], to='orf')

In [23]:
# # Make sure everything translated ok
# t = looks_like_orf(tested['orf'])
# print(tested.loc[~t,])

In [24]:
# tested = tested.loc[t,:]

In [25]:
# tested_orfs = tested['orf'].unique()

In [26]:
# missing = [orf for orf in original_data.index.values if orf not in tested_orfs]
# missing

In [27]:
# original_data = original_data.reindex(index=tested_orfs, fill_value=0)

In [28]:
# original_data.head()

# Prepare the final dataset

In [29]:
data = original_data.copy()

In [30]:
dataset_ids = [113]
datasets = datasets.reindex(index=dataset_ids)

In [31]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [32]:
data.head()

dataset_id,113
data_type,value
orf,
YAL013W,-84.5
YAL016W,-19.5
YAL044C,-67.5
YAL053W,-40.5
YAL056W,-34.0


## Subset to the genes currently in SGD

In [33]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 0


In [34]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,113
,data_type,value
gene_id,orf,
11,YAL013W,-84.5
14,YAL016W,-19.5
42,YAL044C,-67.5
49,YAL053W,-40.5
52,YAL056W,-34.0


# Normalize

In [35]:
data_norm = normalize_phenotypic_scores(data, has_tested=False)

In [36]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [37]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id        113          
data_type       value    valuez
gene_id orf                    
11      YAL013W -84.5 -4.322892
14      YAL016W -19.5 -0.911966
42      YAL044C -67.5 -3.430804
49      YAL053W -40.5 -2.013957
52      YAL056W -34.0 -1.672865

# Print out

In [38]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [42]:
# from IO.save_data_to_db3 import *

In [43]:
# save_data_to_db(data_all, paper_pmid)